In [1]:
import os
import pandas as pd
import numpy as np

from delphi_utils import GeoMapper

os.chdir("../../delphi_utils/data/")

# Basic Utility Usage
Two functions: `add_geocode` and `replace_geocode`.

In [15]:
fips_data = pd.DataFrame({
        "fips":[1123,48253,72003,18181],
        "date":[pd.Timestamp('2018-01-01')]*4,
        "count": [2,1,np.nan,10021],
        "total": [4,1,np.nan,100001]
    })

# Add a new column with the new code
gmpr = GeoMapper()
df = gmpr.add_geocode(fips_data, "fips", "zip")
print(df.head())

# Convert a column with the new code
gmpr = GeoMapper()
df = gmpr.replace_geocode(fips_data, "fips", "zip")
print(df.head())

fips       date  count  total    zip    weight
0  01123 2018-01-01    2.0    4.0  35010  0.461001
1  01123 2018-01-01    2.0    4.0  35072  0.013264
2  01123 2018-01-01    2.0    4.0  35089  0.017661
3  01123 2018-01-01    2.0    4.0  36078  0.113826
4  01123 2018-01-01    2.0    4.0  36255  0.000433
        date    zip     count     total
0 2018-01-01  00602  0.000000  0.000000
1 2018-01-01  00610  0.000000  0.000000
2 2018-01-01  00676  0.000000  0.000000
3 2018-01-01  00677  0.000000  0.000000
4 2018-01-01  35010  0.922001  1.844002


In [16]:
gmpr = GeoMapper()
df = gmpr.replace_geocode(fips_data, "fips", "hrr")
df

,date,hrr,count,total
0,2018-01-01,1,1.772347,3.544694
1,2018-01-01,183,7157.392404,71424.648014
2,2018-01-01,184,2863.607596,28576.351986
3,2018-01-01,382,1.000000,1.000000
4,2018-01-01,7,0.227653,0.455306


In [19]:
df = gmpr.replace_geocode(fips_data, "fips", "hrr")
df2 = gmpr.replace_geocode(fips_data, "fips", "zip")
df2 = gmpr.replace_geocode(df2, "zip", "hrr")
np.allclose(df[['count', 'total']].values, df2[['count', 'total']].values)

True

# Utility Inner Workings

## Deriving a crosswalk
Given two crosswalks, we create a derived crosswalk by merging on the common code.

In [21]:
state_df = pd.read_csv("state_codes_table.csv", dtype={"state_code": str, "state_id": str, "state_name": str})
zip_fips_df = pd.read_csv("zip_fips_table.csv", dtype={"zip": str, "fips": str})
zip_fips_df["state_code"] = zip_fips_df["fips"].str[:2]
zip_state_code_df = zip_fips_df.merge(state_df, on="state_code", how="left").drop(columns=["fips", "state_id", "state_name"])
assert 52 == len(zip_state_code_df.state_code.unique())
zip_state_code_df

,zip,weight,state_code
0,00601,0.994346,72
1,00601,0.005654,72
2,00602,1.000000,72
3,00603,1.000000,72
4,00606,0.948753,72
...,...,...,...
44405,99923,1.000000,02
44406,99925,1.000000,02
44407,99926,1.000000,02
44408,99927,1.000000,02


A weighted one requires a summation.

In [25]:
FIPS_ZIP_OUT_FILENAME = "fips_zip_table.csv"
ZIP_HRR_OUT_FILENAME = "zip_hrr_table.csv"
OUTPUT_DIR = "../../delphi_utils/data"
from os.path import join, isfile

fz_df = pd.read_csv(
    join(OUTPUT_DIR, FIPS_ZIP_OUT_FILENAME),
    dtype={"fips": str, "zip": str, "weight": float},
)
zh_df = pd.read_csv(
    join(OUTPUT_DIR, ZIP_HRR_OUT_FILENAME),
    dtype={"zip": str, "hrr": str},
)

df = (fz_df.merge(zh_df, on="zip", how="left")
          .drop(columns="zip")
          .groupby(["fips", "hrr"])
          .sum()
          .reset_index())
df

,fips,hrr,weight
0,01001,1,0.039105
1,01001,7,0.960895
2,01003,134,0.031998
3,01003,6,0.968002
4,01005,2,0.974360
...,...,...,...
5178,56039,274,0.003804
5179,56039,423,0.996196
5180,56041,423,1.000000
5181,56043,274,1.000000


## Adding a geocode column
Adding a new geocode column is a merge on the left using a matching geocode. Here we translate from zip to fips on some faux data. Since this a merge on the left, invalid ZIP values present in the data, but not present in the crosswalk simply get NAN entries in their columns. Sometimes a "weights" column is added also.

In [27]:
zip_data = pd.DataFrame(
        {
            "zip": ["45140", "45147", "00500", "95616", "95618"],
            "date": pd.date_range("2018-01-01", periods=5),
            "count": [2, np.nan, 20, 100, 21],
            "total": [2, 20, 40, np.nan, 20]
        }
    )
zip_fips_df = pd.read_csv("zip_fips_table.csv", dtype={"zip": str, "fips": str})

data_df = zip_data.merge(zip_fips_df, left_on="zip", right_on="zip", how="left")
data_df

,zip,date,count,total,fips,weight
0,45140,2018-01-01,2.0,2.0,39025,0.523570
1,45140,2018-01-01,2.0,2.0,39061,0.288115
2,45140,2018-01-01,2.0,2.0,39165,0.188315
3,45147,2018-01-02,NaN,20.0,39025,0.938776
4,45147,2018-01-02,NaN,20.0,39061,0.061224
5,00500,2018-01-03,20.0,40.0,NaN,NaN
6,95616,2018-01-04,100.0,NaN,06113,1.000000
7,95618,2018-01-05,21.0,20.0,06095,0.003372
8,95618,2018-01-05,21.0,20.0,06113,0.996628


## Replacing a column
If there are no weights, we just drop the old column and we're done. If there are weights, we multiply the data by the weights and sum over the old codes. A helpful way to think of the operation is a multiplication of the data matrix (row vectors are columns of the dataframe) D by the weights matrix W, D*W. The weights matrix is row-stochastic (i.e. rows sum to 1). 

Note that the aggregation step (i.e. linear combination of source code values) requires a decision for how to handle NA values. We choose to zero-fill them to avoid propagating NAs.

In [28]:
data_df = data_df.drop(columns="zip")

# Multiply and aggregate
data_df[["count", "total"]] = data_df[["count", "total"]].multiply(data_df["weight"], axis=0)
data_df = (data_df.drop("weight", axis=1)
                  .groupby(["date", "fips"])
                  .sum()
                  .reset_index())
data_df

,date,fips,count,total
0,2018-01-01,39025,1.047140,1.047140
1,2018-01-01,39061,0.576229,0.576229
2,2018-01-01,39165,0.376631,0.376631
3,2018-01-02,39025,0.000000,18.775510
4,2018-01-02,39061,0.000000,1.224490
5,2018-01-04,06113,100.000000,0.000000
6,2018-01-05,06095,0.070819,0.067446
7,2018-01-05,06113,20.929181,19.932554


## Building population weights

In [29]:
FIPS_BY_ZIP_POP_URL = (
    "https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt?#"
)
pop_df = pd.read_csv(FIPS_BY_ZIP_POP_URL)

# Create the FIPS column by combining the state and county codes
pop_df["fips"] = pop_df["STATE"].astype(str).str.zfill(2) + pop_df["COUNTY"].astype(
    str
).str.zfill(3)

# Create the ZIP column by adding leading zeros to the ZIP
pop_df["zip"] = pop_df["ZCTA5"].astype(str).str.zfill(5)

# Pare down the dataframe to just the relevant columns: zip, fips, and population
pop_df = pop_df[["zip", "fips", "POPPT"]].rename(columns={"POPPT": "pop"})

pop_df.set_index(
    ["fips", "zip"], inplace=True
)  # can we do without this and resetting index below?
pop_df

pop
fips  zip         
72001 00601  18465
72141 00601    105
72003 00602  41520
72005 00603  54689
72093 00606   6276
...            ...
02198 99923     87
      99925    819
      99926   1460
      99927     94
02275 99929   2338

[44410 rows x 1 columns]

In [31]:
# 2010 Census, corresponds to 308 million population figure
pop_df["pop"].sum()

312462997

## We have updated the FIPS to HRR tables since the last version (James' version)
And they haven't changed by very much.

In [65]:
df_new = GeoMapper().load_crosswalk("fips", "hrr")
df_old = pd.read_csv("https://raw.githubusercontent.com/cmu-delphi/covidcast-indicators/jhu_fix_0824/_delphi_utils_python/delphi_utils/data/fips_hrr_cross.csv?token=AANZ76Q7CUS7REWHRIGNKV27KHH6U", dtype={"fips": str, "hrr": str, "weight": float})
df_old["fips"] = df_old["fips"].str.zfill(5)
df = df_new.groupby(["hrr", "fips"]).sum().reset_index().merge(df_old, on=["fips", "hrr"], how="left")
df.weight_x.sub(df.weight_y).abs().mean()

0.017533503793630306

## We have updated the source for the FIPS to ZIP mapping
Our new one comes from the US Census and the other from [simplemaps.com](https://simplemaps.com/data/us-zips).

In [110]:
df_census = GeoMapper().load_crosswalk("zip", "fips")
df_simplemaps = pd.read_csv("../../data_proc/geomap/uszips.csv")
print(df_simplemaps["population"].sum())

326256148


In [111]:
df_simplemaps["county_weights"] = df_simplemaps["county_weights"].transform(lambda x: list(eval(x).items()))
df_simplemaps = df_simplemaps.explode("county_weights")
df_simplemaps["county_fips"] = df_simplemaps["county_weights"].apply(lambda x: x[0])
df_simplemaps["county_weights"] = df_simplemaps["county_weights"].apply(lambda x: x[1]/100)
df_simplemaps = df_simplemaps.rename(columns={"county_fips": "fips"})
df_simplemaps["zip"] = df_simplemaps["zip"].astype(str).str.zfill(5)
df_simplemaps["fips"] = df_simplemaps["fips"].astype(str).str.zfill(5)
df = df_census.merge(df_simplemaps, on=["zip", "fips"], how="left")

In [62]:
df["weight"].sub(df["county_weights"]).abs().mean()

1.1494991956541422e-05

In [68]:
1 - df["weight"].corr(df["county_weights"])

1.307895680646709e-09

In [120]:
df = df.dropna(subset=["population"])
print(df.groupby("zip")["population"].unique().sum()[0] - df["population"].multiply(df["county_weights"]).sum(),
      df.groupby("zip")["population"].unique().sum()[0] - df["population"].multiply(df["weight"]).sum())

113.4559999704361 147.0


## JHU Time Series Data
This is the data that our JHU indicator pulls to ingest into the API.

In [164]:
df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
important_columns = df[df["UID"] == 84001001].filter(regex="\d{0,2}/\d{0,2}/20").columns.to_list() + ["UID"]
df = df[important_columns]
date_columns = set(df.columns) - set(["UID"])
df = df.melt(id_vars=['UID'], value_vars=date_columns).rename(columns={"variable": "date"})
df["date"] = pd.to_datetime(df["date"])

In [169]:
df_fips = GeoMapper().replace_geocode(df, "jhu_uid", "fips", "UID")
df_msa = GeoMapper().add_geocode(df_fips, "fips", "msa")

In [185]:
df_fipspop = GeoMapper().load_crosswalk("fips", "pop")
df_fipsmsa = GeoMapper().load_crosswalk("fips", "msa")
df = df_fipspop.merge(df_fipsmsa, on="fips", how="left")
df.groupby("msa")["pop"].sum().value_counts().sort_index()

56210       1
60368       1
60987       1
62036       1
73827       1
           ..
6773350     1
7176780     1
9525478     1
13262011    1
19299786    1
Name: pop, Length: 392, dtype: int64

## JHU UID and FIPS Codes
There are a number by-hand modification we have to make to the JHU UID -> FIPS map (see [here](https://cmu-delphi.github.io/delphi-epidata/api/covidcast-signals/jhu-csse.html#geographical-exceptions)). 

To get started, let's compare their population data to population data we use for FIPS codes. We obtain a 5.6% difference, that's not bad.

In [32]:
JHU_FIPS_URL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
df = pd.read_csv(JHU_FIPS_URL)
df2 = pd.read_csv("fips_pop.csv")
merged = df2.merge(df, left_on="fips", right_on="FIPS", how="left")
((merged["pop"] - merged["Population"]).abs()/merged["Population"]).mean()

0.05688630188845362

Here we verify the hand-additions. 

Even though there are multiple JHU UIDs for some FIPS because of the splitting, the extra data is fine because it will likely generate NAs (for example, the Dukes and Nantucket counties are reported together, which is why they have a separate UID; we population weight split this in the two respective FIPS codes).

In [35]:
hand_additions = pd.DataFrame(
    [
        # Split aggregation of Dukes and Nantucket, Massachusetts
        {"jhu_uid": 84070002, "fips": "25007", "weight": 16/26}, # Population: 16535
        {"jhu_uid": 84070002, "fips": "25019", "weight": 10/26}, # 10172
        # Kansas City, Missouri
        {"jhu_uid": 84070003, "fips": "29095", "weight": 674158 / 1084897}, # Population: 674158 
        {"jhu_uid": 84070003, "fips": "29165", "weight": 89322 / 1084897}, # 89322
        {"jhu_uid": 84070003, "fips": "29037", "weight": 99478 / 1084897}, # 99478
        {"jhu_uid": 84070003, "fips": "29047", "weight": 221939 / 1084897}, # 221939
        # Kusilvak, Alaska
        {"jhu_uid": 84002158, "fips": "02270", "weight": 1.0},
        # Split aggregation of New York County (populations given at JHU documentation)
        {"jhu_uid": 84036061, "fips": "36005", "weight": 1418207/8336817}, # Population: 1,418,207
        {"jhu_uid": 84036061, "fips": "36047", "weight": 2559903/8336817}, # 2,559,903
        {"jhu_uid": 84036061, "fips": "36061", "weight": 1628706/8336817}, # 1,628,706
        {"jhu_uid": 84036061, "fips": "36081", "weight": 2253858/8336817}, # 2,253,858
        {"jhu_uid": 84036061, "fips": "36085", "weight": 476143/8336817}, # 476,143
    ]
)

jhu_df = pd.read_csv(JHU_FIPS_URL, dtype={"UID": str, "FIPS": str})
jhu_df = jhu_df.query("Country_Region == 'US'")
jhu_df = (
    jhu_df[["UID", "FIPS"]]
    .rename(columns={"UID": "jhu_uid", "FIPS": "fips"})
    .dropna(subset=["fips"])
)

# FIPS Codes that are just two long should be zero filled.
# These are Guam (66), Northern Mariana Islands (69), Virgin Islands (78),
# and Puerto Rico (72).
fips_st = jhu_df["fips"].str.len() <= 2
jhu_df.loc[fips_st, "fips"] = jhu_df.loc[fips_st, "fips"].astype(str).str.zfill(5)

# Drop the JHU UIDs that were hand-modified
dup_ind = jhu_df["jhu_uid"].isin(hand_additions["jhu_uid"].values)
jhu_df.drop(jhu_df.index[dup_ind], inplace=True)

# Drop the FIPS codes in JHU that were hand-modified
dup_ind = jhu_df["fips"].isin(hand_additions["fips"].values)
jhu_df.drop(jhu_df.index[dup_ind], inplace=True)

# Add weights of 1.0 to everything not in hand additions, then merge in hand-additions
# Finally, zero fill FIPS
jhu_df["weight"] = 1.0
jhu_df = pd.concat((jhu_df, hand_additions))
jhu_df["fips"] = jhu_df["fips"].astype(int).astype(str).str.zfill(5)
jhu_df

,jhu_uid,fips,weight
750,16,00060,1.000000
751,316,00066,1.000000
752,580,00069,1.000000
753,850,00078,1.000000
754,630,00072,1.000000
...,...,...,...
7,84036061,36005,0.170114
8,84036061,36047,0.307060
9,84036061,36061,0.195363
10,84036061,36081,0.270350


We should have no FIPS codes with multiple entries, because this would imply multiple JHU UIDs.

In [36]:
new_df = jhu_df.groupby("fips").count().reset_index()
new_df[new_df["jhu_uid"] >= 2]["fips"].to_list()

[]

## Verifying JHU reporting
Puerto Rico reports cases at the municipality level, while deaths are reported at the commonwealth level in the Unassigned UID (see [here](https://github.com/CSSEGISandData/COVID-19/issues/2889)).

In [123]:
import pandas as pd
import numpy as np

def convert_time(t):
    """Shape date into m/d/yy format."""
    return str(int(t.strftime("%m"))) + "/" + str(int(t.strftime("%d"))) +  "/" + t.strftime("%y")

# Load JHU deaths
df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

# Get last two months
date_cols = [convert_time(t) for t in pd.date_range("06-01-2020", "08-01-2020")]
puerto_rico_uids = [63072999]

# Check for non-zero deaths
print("Average deaths June 1st to August 1st: ", df[df["UID"].isin(puerto_rico_uids)][date_cols].to_numpy().mean())

# Load JHU confirmed
df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")

# Get last two months
date_cols = [convert_time(t) for t in pd.date_range("06-01-2020", "08-01-2020")]
puerto_rico_uids = [int("630" + str(i)) for i in range(72001, 72200)]

# Check for non-zero cases
print("Average confirmed cases June 1st to August 1st: ", df[df["UID"].isin(puerto_rico_uids)][date_cols].to_numpy().mean())

Average deaths June 1st to August 1st:  162.74193548387098
Average confirmed cases June 1st to August 1st:  107.18114143920596


In [122]:
# Load JHU confirmed
df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")

# Get last two months
date_cols = [convert_time(t) for t in pd.date_range("06-01-2020", "08-01-2020")]
puerto_rico_uids = [63072999]

# Check for non-zero cases
print("Any confirmed cases June 1st to August 1st? ",np.any(df[df["UID"].isin(puerto_rico_uids)][date_cols].to_numpy()>0))
print("Average confirmed cases June 1st to August 1st: ", df[df["UID"].isin(puerto_rico_uids)][date_cols].to_numpy().mean())

Any confirmed cases June 1st to August 1st?  True
Average confirmed cases June 1st to August 1st:  323.0806451612903


In [129]:
puerto_rico_uids = [int("630" + str(i)) for i in range(72001, 72200)]
df[df["UID"].isin(puerto_rico_uids)][date_cols].sum()

6/1/20      3749
6/2/20      3802
6/3/20      3888
6/4/20      4364
6/5/20      4481
           ...  
7/28/20    14871
7/29/20    15075
7/30/20    15582
7/31/20    15811
8/1/20     16802
Length: 62, dtype: int64

Let's try New York. The only county that should report anything is "36061", which aggregates all the NY City counties.

In [38]:
df_ny = df[df["FIPS"].isin(["36005", "36047", "36061", "36081", "36085"])]
df_ny

,FIPS,Deaths
2527,36061,23658


What about Dukes?

In [ ]:
df[df["FIPS"].isin(["36005", "36047", "36061", "36081", "36085"])]

Let's look at Utah. It's not reported at the FIPS level, but instead is repoted in an aggregate.

In [133]:
df[df["UID"].isin([84049001])] # Beaver county, no data

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
2955,84049001,US,USA,840,49001.0,Beaver,Utah,US,38.356571,-113.234223,...,0,0,0,0,0,0,0,0,0,0


In [134]:
df[df["UID"].isin([84070015])] # Bear River aggregate

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
2954,84070015,US,USA,840,NaN,Bear River,Utah,US,41.521068,-113.083282,...,2372,2384,2393,2398,2405,2413,2427,2443,2464,2482


In [131]:
df_ut

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
2954,84070015,US,USA,840,NaN,Bear River,Utah,US,41.521068,-113.083282,...,2372,2384,2393,2398,2405,2413,2427,2443,2464,2482
